## 1. east text detection + pytesseract(text recognition)
- east text detection 참고 : https://pyimagesearch.com/2018/08/20/opencv-text-detection-east-text-detector/

In [ ]:
import cv2
import numpy as np
from imutils.object_detection import non_max_suppression
import pytesseract

min_confidence = 0.05
file_name = "test7.jpg"

east_decorator = 'frozen_east_text_detection.pb'

frame_size = 320
padding = 0.15

def textRead(image):  #tessaract 이용
    # apply Tesseract v4 to OCR 
    config = ("-l kor+eng --oem 1 --psm 12")
    text = pytesseract.image_to_string(image, config=config)
    # display the text OCR'd by Tesseract
    print("OCR TEXT : {}\n".format(text))
    
    # strip out non-ASCII text (문자, 숫자만 필터링)
    text = "".join([c if c.isalnum() else "" for c in text]).strip()
    print("TEXT : {}\n".format(text))
    return text

def textROI(image): #east text detection 이용
    # load the input image and grab the image dimensions
    orig = image.copy()
    (origH, origW) = image.shape[:2]
 
    # set the new width and height and then determine the ratio in change
    rW = origW / float(frame_size)
    rH = origH / float(frame_size)
    newW = int(origW / rH)
    center = int(newW / 2)
    start = center - int(frame_size / 2)
 
    # resize the image and grab the new image dimensions
    image = cv2.resize(image, (newW, frame_size))  
    scale_image = image[0:frame_size, start:start+frame_size]
    (H, W) = scale_image.shape[:2]

#     cv2.imshow("orig", orig)
#     cv2.imshow("resize", image)
#     cv2.imshow("scale_image", scale_image)
    
    # define the two output layer names for the EAST detector model 
    layerNames = [
            "feature_fusion/Conv_7/Sigmoid",
            "feature_fusion/concat_3"]
    
    # load the pre-trained EAST text detector
    net = cv2.dnn.readNet(east_decorator)

    # construct a blob from the image 
    blob = cv2.dnn.blobFromImage(image, 1.0, (frame_size, frame_size),
            (123.68, 116.78, 103.94), swapRB=True, crop=False)
    net.setInput(blob)
    (scores, geometry) = net.forward(layerNames)

    (numRows, numCols) = scores.shape[2:4]
    rects = []
    confidences = []

    # loop over the number of rows
    for y in range(0, numRows):
        # extract the scores (probabilities)
        scoresData = scores[0, 0, y]
        xData0 = geometry[0, 0, y]
        xData1 = geometry[0, 1, y]
        xData2 = geometry[0, 2, y]
        xData3 = geometry[0, 3, y]
        anglesData = geometry[0, 4, y]

        # loop over the number of columns
        for x in range(0, numCols):

                if scoresData[x] < min_confidence:
                        continue

                (offsetX, offsetY) = (x * 4.0, y * 4.0)

                angle = anglesData[x]
                cos = np.cos(angle)
                sin = np.sin(angle)

                h = xData0[x] + xData2[x]
                w = xData1[x] + xData3[x]

                endX = int(offsetX + (cos * xData1[x]) + (sin * xData2[x]))
                endY = int(offsetY - (sin * xData1[x]) + (cos * xData2[x]))
                startX = int(endX - w)
                startY = int(endY - h)

                rects.append((startX, startY, endX, endY))
                confidences.append(scoresData[x])
    
    # apply non-maxima suppression 
    boxes = non_max_suppression(np.array(rects), probs=confidences)

    # initialize the list of results
    results = []

    # loop over the bounding boxes
    for (startX, startY, endX, endY) in boxes:

            startX = int(startX * rW)
            startY = int(startY * rH)
            endX = int(endX * rW)
            endY = int(endY * rH)
            
#             cv2.rectangle(img_copy, (startX, startY), (endX, endY), (0, 255, 0), 2)

            dX = int((endX - startX) * padding)
            dY = int((endY - startY) * padding)

            startX = max(0, startX - dX)
            startY = max(0, startY - dY)
            endX = min(origW, endX + (dX * 2))
            endY = min(origH, endY + (dY * 2))

            # extract the actual padded ROI
            cv2.rectangle(img_copy, (startX, startY), (endX, endY), (0, 255, 0), 2)
            text = textRead(orig[startY:endY, startX:endX])

            #cv2.rectangle(img_copy, (x+startX, y+startY), (x+endX, y+endY), (0, 255, 0), 2)

            cv2.putText(img_copy, text, (startX, startY-10),
                cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
#             #return ([startX, startY, endX, endY], orig[startY:endY, startX:endX])

# Loading image
img = cv2.imread(file_name)
img_copy = img.copy()
textROI(img)
# ([startX, startY, endX, endY], text_image) = textROI(img)

# cv2.rectangle(img_copy, (startX, startY), (endX, endY), (0, 255, 0), 2)

# show the output image
cv2.imshow("OCR Text Recognition", img_copy)

cv2.waitKey(0)
cv2.destroyAllWindows()

## 2. easyOCR (image)
- text detection + text recognition 한번에 함
- tesseract보다 정확도 높음

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np

In [ ]:
IMAGE_PATH = 'test__.jpg'

reader = easyocr.Reader(['en']) #한글은 'ko'
result = reader.readtext(IMAGE_PATH)
result

In [ ]:
top_left = tuple(result[0][0][0])
bottom_right = tuple(result[0][0][2])
text = result[0][1]
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
img = cv2.imread(IMAGE_PATH)
img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
img = cv2.putText(img,text,top_left, font, 2,(255,255,255),3,cv2.LINE_AA)
plt.imshow(img)
plt.show()

In [ ]:
img = cv2.imread(IMAGE_PATH)
spacer = 100
for detection in result: 
    top_left = tuple(detection[0][0])
    bottom_right = tuple(detection[0][2])
    text = detection[1]
    img = cv2.rectangle(img,top_left,bottom_right,(0,255,0),3)
    img = cv2.putText(img,text,(20,spacer), font, 2,(0,255,0),2,cv2.LINE_AA)
    spacer+=60
    
plt.imshow(img)
plt.show()

## 3. easyOCR (real time)

In [ ]:
!pip install easyocr

In [ ]:
import easyocr
import cv2
from matplotlib import pyplot as plt
import numpy as np
import time

In [ ]:
reader = easyocr.Reader(['en', 'ko'], gpu=True) #gpu 안쓸거면 gpu=True 부분 그냥 지우기
vid = cv2.VideoCapture(0)
if not vid.isOpened:
    print('Error opening video')
    exit(0)
skip_frame = True

while(True):
#     a = time.time()
    ret, img = vid.read()
    if img is None:
        print('No more frame')
        vid.release()
        break
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    result = reader.readtext(gray)
    text = ""
    
    for res in result:
        text += res[1] + " "
    cv2.putText(img, text, (50, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (50, 50, 255), 2)
    
#     ## FPS
#     b = time.time()
#     fps = 1/(b-a)
#     cv2.line(img, (20, 25), (127, 25), [85, 45, 255], 30)
#     cv2.putText(img, text, (50, 70), xv2.FONT_HERSHEY_SCRIPT_SIMPLEX, 1, (50, 50, 255), 2)
    cv2.imshow("result", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
#     print(fps)
    print(text)

vid.release()
cv2.destroyAllWindows()